Simulation

In [ ]:
import numpy as np
from control import lqr, ss, forced_response

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Define system parameters
m = 0.0318  # Mass of the pendulum (kg)
M = 0.3333  # Mass of the cart (kg)
l = 0.316/2  # Length of the pendulum (m)
g = 9.81  # Gravitational acceleration (m/s^2)
I = 0.0085*(0.0098**2+0.0379**2)/12 + m*((l*2)**22)/3  # Moment of inertia (kg*m^2)

# Damping coefficient (not used in LQR but can be included for more realistic dynamics)
a1 = 0.0185
c1 = 2*a1*I      # Viscous friction of pendulum 1 (rotational) (Nms/rad)

alpha = 12.2;      # Carriage slope (deg)
xdotss = 0.4852;   # Terminal velocity (m/s)
c = (M+m)*g*np.sin(alpha*np.pi/180)/xdotss;   # Damping / Viscous Friction (kg/s)

# State-space matrices
denom = (M+m)*(m*l**2+I)

A = np.array([
    [0, 1, 0, 0],
    [0, -(m*l**2+I)*c/denom, m**2*l**2*g/denom, m*l*c1/denom],
    [0, 0, 0, 1],
    [0, -m*l*c/denom, (M+m)*m*l*g/denom, (M+m)*c1/denom]
])
B = np.array([[0], [(m*l**2+I)/denom], [0], [m*l/denom]])
C = np.eye(4)
D = np.zeros((4, 1))

# Create state-space system
system = ss(A, B, C, D)

# LQR controller design
Q = np.diag([48000, 0, 100, 0])  # State cost matrix
R = np.array([[1]])        # Control effort cost
K, _, _ = lqr(A, B, Q, R)

# Simulation parameters
T = np.linspace(0, 10, 1000)  # 10 seconds, 1000 points
X0 = [0.0, 0.0, 0.1, 0.0]     # Initial state: small angle offset

# Reference (desired state)
ref = np.zeros((4, len(T)))   # Track zero for all states

# Simulate closed-loop system
def lqr_control(t, x):
    # u = -Kx
    u = -K @ x.reshape(-1, 1)
    return u.item()

X = np.zeros((4, len(T)))
X[:, 0] = X0
U = np.zeros(len(T))

dt = T[1] - T[0]
for i in range(1, len(T)):
    u = lqr_control(T[i-1], X[:, i-1])
    U[i-1] = u
    # x_dot = Ax + Bu
    x_dot = A @ X[:, i-1] + B.flatten() * u
    X[:, i] = X[:, i-1] + x_dot * dt

# Plot all states on the same graph
plt.figure(figsize=(10, 6))
plt.plot(T, X[0, :], label='Cart Position (m)')
plt.plot(T, X[1, :], label='Cart Velocity (m/s)')
plt.plot(T, X[2, :], label='Pendulum Angle (rad)')
plt.plot(T, X[3, :], label='Pendulum Angular Velocity (rad/s)')
plt.xlabel('Time (s)')
plt.ylabel('State Value')
plt.title('Inverted Pendulum States Over Time')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()